# MLlib事始め - 二値分類の例

このチュートリアルは、Apache SparkのMLlibに慣れることを目的としています。ここでは、デモグラフィック情報に基づいて、年収が5万ドル以上か否かを分類する二値分類問題に取り組みます。データセットは、[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult)にあるものを利用します。既にこちらのデータはDatabricksランタイムに格納されています。このノートブックでは、データ処理、機械学習パイプライン、機械学習アルゴリズムなどのMLlibの機能をデモンストレーションします。

このノートブックでは以下のステップを踏みます：

1. データセットの読み込み
1. 特徴量の前処理
1. モデルの定義
1. パイプラインの構築
1. モデルの評価
1. ハイパーパラメーターのチューニング
1. 予測の実行、モデル性能の評価

**要件**
- Databricks MLランタイム7.0以上が必要です。

## Step 1. データセットの読み込み

データの最初の行を見るためにDatabricksのユーティリティを使います

In [0]:
%fs head --maxBytes=1024 databricks-datasets/adult/adult.data

[Truncated to first 1024 bytes]
39, State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K
50, Self-emp-not-inc, 83311, Bachelors, 13, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 13, United-States, <=50K
38, Private, 215646, HS-grad, 9, Divorced, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States, <=50K
53, Private, 234721, 11th, 7, Married-civ-spouse, Handlers-cleaners, Husband, Black, Male, 0, 0, 40, United-States, <=50K
28, Private, 338409, Bachelors, 13, Married-civ-spouse, Prof-specialty, Wife, Black, Female, 0, 0, 40, Cuba, <=50K
37, Private, 284582, Masters, 14, Married-civ-spouse, Exec-managerial, Wife, White, Female, 0, 0, 40, United-States, <=50K
49, Private, 160187, 9th, 5, Married-spouse-absent, Other-service, Not-in-family, Black, Female, 0, 0, 16, Jamaica, <=50K
52, Self-emp-not-inc, 209642, HS-grad, 9, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 45, United-States, >50K
31, Private, 45781, Masters,

データセットにはカラム名が含まれていないため、カラム名とデータタイプを指定するスキーマを作成します

In [0]:
schema = """`age` DOUBLE,
`workclass` STRING,
`fnlwgt` DOUBLE,
`education` STRING,
`education_num` DOUBLE,
`marital_status` STRING,
`occupation` STRING,
`relationship` STRING,
`race` STRING,
`sex` STRING,
`capital_gain` DOUBLE,
`capital_loss` DOUBLE,
`hours_per_week` DOUBLE,
`native_country` STRING,
`income` STRING"""

dataset = spark.read.csv("/databricks-datasets/adult/adult.data", schema=schema)

データセットをランダムにトレーニングデータとテストデータに分割します。再現性確保のために乱数のシードを設定しています。

あらゆる前処理を実行する前にデータを分割すべきです。これにより、モデルを評価する際、テストデータが未知のデータに近い状態を維持することができます。

In [0]:
trainDF, testDF = dataset.randomSplit([0.8, 0.2], seed=42)
print("トレーニングデータ:", trainDF.cache().count()) # 何回かトレーニングするのでデータをキャッシュします
print("テストデータ:", testDF.count())

トレーニングデータ: 26076
テストデータ: 6485

データを確認しましょう

In [0]:
display(trainDF)

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
17.0,?,34019.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K
17.0,?,34088.0,12th,8.0,Never-married,?,Own-child,White,Female,0.0,0.0,25.0,United-States,<=50K
17.0,?,47407.0,11th,7.0,Never-married,?,Own-child,White,Male,0.0,0.0,10.0,United-States,<=50K
17.0,?,48703.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
17.0,?,48751.0,11th,7.0,Never-married,?,Own-child,Black,Female,0.0,0.0,40.0,United-States,<=50K
17.0,?,67808.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K
17.0,?,86786.0,10th,6.0,Never-married,?,Own-child,White,Female,0.0,0.0,40.0,United-States,<=50K
17.0,?,89870.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K
17.0,?,94366.0,10th,6.0,Never-married,?,Other-relative,White,Male,0.0,0.0,6.0,United-States,<=50K
17.0,?,103810.0,12th,8.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K


`hours_per_week`の数の分布はどうなっているでしょうか？

In [0]:
display(trainDF.select("hours_per_week").summary())

summary,hours_per_week
count,26076
mean,40.4284782942169
stddev,12.404569739132008
min,1.0
25%,40.0
50%,40.0
75%,45.0
max,99.0


`education`はどうなっているでしょうか？

In [0]:
display(trainDF
        .groupBy("education")
        .count()
        .sort("count", ascending=False))

education,count
HS-grad,8408
Some-college,5860
Bachelors,4255
Masters,1388
Assoc-voc,1102
11th,958
Assoc-acdm,845
10th,748
7th-8th,510
Prof-school,465


## バックグラウンド: Transformers、estimators、pipelines

本ノートブックで説明するMLlibの機械学習における重要な3つのコンセプトは、**Transformers**、**Estimators**、そして、**Pipelines**です。
<br>
- **Transformer**: データフレームをインプットとして新たなデータフレームを返却します。Transformersは、データから学習は行わず、モデル学習のためのデータを準備するか、学習したMLlibモデルで予測を行うために、単にルールベースの変換処理を適用します。`.transform()`メソッドでtransformerを呼び出すことができます。

- **Estimator**: `.fit()`メソッドを用いてデータフレームからパラメーターを学習(fit)し、モデルを返却します。モデルはtransformerです。

- **Pipeline**: 複数のステップを容易に実行できるように単一のワークフローにまとめます。機械学習モデル作成には、多くのケースで異なるステップが含まれ、それらを繰り返す必要があります。パイプラインを用いることでこのプロセスを自動化することができます。

詳細はこちらを参照ください:
[ML Pipelines(英語)](https://spark.apache.org/docs/latest/ml-pipeline.html#ml-pipelines)

## Step 2. 特徴量の前処理

このノートブックのゴールは、データセットに含まれる特徴量(教育レベル、既婚・未婚、職業など)から`income`のレベルを予測するというものです。この最初のステップは、MLlibで利用できるように特徴量を操作、前処理することです。

### カテゴリー変数を数値に変換する

線形回帰、ロジスティック回帰などの学習アルゴリズムでは、特徴量が数値である必要があります。上記の成人データセットでは、教育、職業、既婚・未婚のデータがカテゴリー変数となっています。

以下のコードでは、カテゴリー変数を0か1のみを取る数値変数に変換するために、どのように`StringIndexer`と`OneHotEncoder`を使用するのかを説明します。

- `StringIndexer`は、文字列のカラムをラベルのインデックスに変換します。例えば、"red"、"blue"、"green"をそれぞれ0、1、2に変換します。 
- `OneHotEncoder`は、カテゴリー変数のインデックスを二進数のベクトルにマッピングします。当該レコードのカテゴリー変数のインデックスに該当するベクトルの要素に"1"が割り当てられます。

SparkにおけるOne-hotエンコーディングは2段階のプロセスとなります。最初にStringIndexerを使い、OneHotEncoderを呼び出します。以下のコードブロックでは、StringIndexerとOneHotEncoderを定義しますが、データにはまだ適用しません。

詳細はこちらを参照ください:   
- [StringIndexer(英語)](http://spark.apache.org/docs/latest/ml-features.html#stringindexer)   
- [OneHotEncoder(英語)](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

categoricalCols = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex"]

# 以下の２行はestimatorとなります。後ほどデータセットを変換する際に適用することになる関数を返却します。
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols]) 
encoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols]) 

# ラベルとなるカラム("income")も("<=50K"、">50K")の二つの値をとる文字列のカラムとなります。
# こちらもStringIndexerを使って数値に変換します。
labelToIndex = StringIndexer(inputCol="income", outputCol="label")

このノートブックでは、特徴量エンジニアリングとモデル構築のステップ全てを一つのパイプラインにまとめます。ただ、その前に上のコードブロックで構築した`stringIndexer`estimatorを適用することでestimatorやtransformerがどのように動作するのかを詳しく見てみましょう。

データセットを変換する`StringIndexerModel`を返却するように`.fit()`メソッドを呼び出します。

そして、`StringIndexerModel`の`.transform()`メソッドを呼び出すことで、カラムが追加された新たなデータフレームが返却されます。必要であれば、表示結果を右にスクロールして追加されたカラムを参照してください。

詳細はこちらを参照ください: [StringIndexerModel(英語)](https://spark.apache.org/docs/latest/api/java/org/apache/spark/ml/feature/StringIndexerModel.html)

In [0]:
stringIndexerModel = stringIndexer.fit(trainDF)
display(stringIndexerModel.transform(trainDF))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,educationIndex,raceIndex,occupationIndex,relationshipIndex,workclassIndex,marital_statusIndex,sexIndex
17.0,?,34019.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,34088.0,12th,8.0,Never-married,?,Own-child,White,Female,0.0,0.0,25.0,United-States,<=50K,11.0,0.0,7.0,2.0,3.0,1.0,1.0
17.0,?,47407.0,11th,7.0,Never-married,?,Own-child,White,Male,0.0,0.0,10.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,48703.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,1.0
17.0,?,48751.0,11th,7.0,Never-married,?,Own-child,Black,Female,0.0,0.0,40.0,United-States,<=50K,5.0,1.0,7.0,2.0,3.0,1.0,1.0
17.0,?,67808.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,86786.0,10th,6.0,Never-married,?,Own-child,White,Female,0.0,0.0,40.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,1.0
17.0,?,89870.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0
17.0,?,94366.0,10th,6.0,Never-married,?,Other-relative,White,Male,0.0,0.0,6.0,United-States,<=50K,7.0,0.0,7.0,5.0,3.0,1.0,0.0
17.0,?,103810.0,12th,8.0,Never-married,?,Own-child,White,Male,0.0,0.0,40.0,United-States,<=50K,11.0,0.0,7.0,2.0,3.0,1.0,0.0


### 全ての特徴量カラムを一つの特徴量ベクトルにまとめます

多くのMLlibアルゴリズムでは、入力として単一の特徴量カラムが必要となります。それぞれの行の特徴量カラムは、予測に用いる特徴量に対応するベクトルを保持します。

MLlibは、一連のカラムから単一のベクトルカラムを作成する`VectorAssembler`transformerを提供します。

下のコードブロックではどのようにVectorAssemblerを使用するのかを説明します。

詳細はこちらを参照ください: [VectorAssembler(英語)](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)

In [0]:
from pyspark.ml.feature import VectorAssembler

# ここには、データセットの数値カラムとone-hotエンコードされた２値のベクトル両方が含まれます。
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "OHE" for c in categoricalCols] + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

## Step 3. モデルの定義

本ノートブックでは[ロジスティック回帰(英語)](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)モデルを使います。

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label", regParam=1.0)

## Step 4. パイプラインの構築

`Pipeline`は、transformers、estimatorsが含まれる順番付きのリストです。データセットに適用する変換処理の再現性を確保し、自動化するために、パイプラインを定義することができます。

`StringIndexer`で見たのと同様に、`Pipeline`もestimatorです。`pipeline.fit()`メソッドが、transformerである`PipelineModel`を返却します。

詳細はこちらを参照ください:
[Pipelines(英語)](https://spark.apache.org/docs/latest/ml-pipeline.html#ml-pipelines)

In [0]:
from pyspark.ml import Pipeline

# これまでに作成したステージを組み合わせてパイプラインを定義します
pipeline = Pipeline(stages=[stringIndexer, encoder, labelToIndex, vecAssembler, lr])

# パイプラインモデルを定義します
pipelineModel = pipeline.fit(trainDF)

# テストデータセットにパイプラインモデルを適用します
predDF = pipelineModel.transform(testDF)

モデルによる予測結果を表示します。`features`カラムは、one-hotエンコーディングを実行した後、多くのケースで要素のほとんどが0となる[sparse vector(英語)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.linalg.SparseVector.html#pyspark.ml.linalg.SparseVector)となります。

In [0]:
display(predDF.select("features", "label", "prediction", "probability"))

features,label,prediction,probability
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 41643.0, 7.0, 15.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9062474976435612, 0.09375250235643884))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 15, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 64785.0, 6.0, 30.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8927691288853359, 0.10723087111466412))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 80077.0, 7.0, 20.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9041097206748698, 0.09589027932513017))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 104025.0, 7.0, 18.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8952738661074805, 0.1047261338925195))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 15, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 139183.0, 6.0, 15.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9087696046250316, 0.09123039537496835))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 8, 24, 36, 45, 49, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 148769.0, 9.0, 40.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8861377407390763, 0.11386225926092375))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 170320.0, 7.0, 8.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9089516898879618, 0.09104831011203818))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 198797.0, 7.0, 20.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8942281267283654, 0.10577187327163462))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 19, 24, 36, 45, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 241021.0, 8.0, 40.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.8945663579567187, 0.10543364204328132))"
"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 49, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 250541.0, 7.0, 8.0))",0.0,0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9099365057082249, 0.09006349429177507))"


## Step 5. モデルの評価

`display`コマンドにはROCカーブを表示するオプションが組み込まれています。

In [0]:
display(pipelineModel.stages[-1], predDF.drop("prediction", "rawPrediction", "probability"), "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.5139985144933663
0.0,0.038461538461538464,0.5139985144933663
0.0,0.07692307692307693,0.509717350710216
0.0,0.11538461538461539,0.5001046759516439
0.0,0.15384615384615385,0.4996941428384779
0.010526315789473684,0.15384615384615385,0.49745087514830555
0.010526315789473684,0.19230769230769232,0.4908963582806457
0.010526315789473684,0.23076923076923078,0.46209046828352585
0.010526315789473684,0.2692307692307692,0.4355582066136938
0.010526315789473684,0.3076923076923077,0.4317969906768171


モデル評価において、ROCカーブのAUC(Area Under the Curve)を計算するために`BinaryClassificationEvaluator`を用い、精度を評価するために`MulticlassClassificationEvaluator`を用います。

詳細はこちらを参照ください:
- [BinaryClassificationEvaluator(英語)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html#binaryclassificationevaluator)  
- [MulticlassClassificationEvaluator(英語)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html#multiclassclassificationevaluator)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

bcEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print(f"Area under ROC curve: {bcEvaluator.evaluate(predDF)}")

mcEvaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print(f"Accuracy: {mcEvaluator.evaluate(predDF)}")

Area under ROC curve: 0.8832451799218509
Accuracy: 0.7680801850424056

## Step 6. ハイパーパラメーターのチューニング

MLlibはハイパーパラメーターチューニングと交差検証(cross validation)の手段を提供します。
- ハイパーパラメータチューニングにおいては、`ParamGridBuilder`を用いることで、モデルのハイパーパラメーターの探索空間を定義できます。
- 交差検証においては、`CrossValidator`を用いることで、estimator(入力データセットに適用するパイプライン)、evaluator、ハイパーパラメーターの探索空間、交差検証のフォールド数を定義できます。

詳細はこちらを参照ください:
- [交差検証（クロスバリデーション）とは？合わせてグリッドサーチに関しても学ぼう！ \| AI Academy Media](https://aiacademy.jp/media/?p=263)
- [Model selection using cross-validation(英語)](https://spark.apache.org/docs/latest/ml-tuning.html)  
- [ParamGridBuilder(英語)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html#paramgridbuilder)  
- [CrossValidator(英語)](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html#crossvalidator)

モデルをチューニングするために、`ParamGridBuilder`と`CrossValidator`を使用します。本例においては、`CrossValidator`での検証において、3種類の`regParam`、3種類の`elasticNetParam`から生成される、3 x 3 = 9のハイパーパラメーターの組み合わせを使用します。

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .build())

MLlibの`CrossValidator`を呼び出した際、Databricksは[MLflow](https://mlflow.org/)を用いて、自動的に全てのランを追跡します。MLflowのUI([AWS](https://docs.databricks.com/applications/mlflow/index.html)|[Azure](https://docs.microsoft.com/azure/databricks/applications/mlflow/))を用いて、構築したモデルを比較することができます。下のセルの実行後、画面右上にある**Experiment**ボタンを押してみてください。

本例では、作成したパイプラインをestimatorとします。

In [0]:
# 3フォールドのCrossValidatorを作成
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=bcEvaluator, numFolds=3, parallelism = 4)

# 交差検証の実施。交差検証からベストなモデルを得るために処理に数分かかる場合があります。
cvModel = cv.fit(trainDF)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

## Step 7. 予測の実行、モデル性能の評価

テストデータセットに対する予測を行うために、交差検証によって特定されたベストモデルを用い、AUCによるモデルの性能を評価します。

In [0]:
# テストデータセットに対する予測を行うために、交差検証によって特定されたベストモデルを使用
cvPredDF = cvModel.transform(testDF)

# AUCと精度を用いてモデルの性能を評価 
print(f"Area under ROC curve: {bcEvaluator.evaluate(cvPredDF)}")
print(f"Accuracy: {mcEvaluator.evaluate(cvPredDF)}")

Area under ROC curve: 0.902341381999495
Accuracy: 0.8450269853508096

予測結果のデータセットを見てみます。`prediction`カラムの値が0の場合、`<=50K`、1の場合`>50K`と予測したことを意味します。

In [0]:
display(cvPredDF)

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,educationIndex,raceIndex,occupationIndex,relationshipIndex,workclassIndex,marital_statusIndex,sexIndex,relationshipOHE,sexOHE,raceOHE,marital_statusOHE,educationOHE,workclassOHE,occupationOHE,label,features,rawPrediction,probability,prediction
17.0,?,41643.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,15.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,1.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 6, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 14, indices -> List(7), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 41643.0, 7.0, 15.0))","Map(vectorType -> dense, length -> 2, values -> List(6.919713883878681, -6.919713883878681))","Map(vectorType -> dense, length -> 2, values -> List(0.999012862787693, 9.87137212307032E-4))",0.0
17.0,?,64785.0,10th,6.0,Never-married,?,Own-child,White,Male,0.0,0.0,30.0,United-States,<=50K,7.0,0.0,7.0,2.0,3.0,1.0,0.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 6, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(7), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 14, indices -> List(7), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 59, indices -> List(3, 15, 24, 36, 45, 48, 52, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 64785.0, 6.0, 30.0))","Map(vectorType -> dense, length -> 2, values -> List(6.122489863137553, -6.122489863137553))","Map(vectorType -> dense, length -> 2, values -> List(0.9978118097594444, 0.0021881902405556097))",0.0
17.0,?,80077.0,11th,7.0,Never-married,?,Own-child,White,Female,0.0,0.0,20.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,1.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 6, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 15, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(3), values -> List(1.0))","Map(vectorType -> sparse, length -> 14, indices -> List(7), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 59, indices -> List(3, 13, 24, 36, 45, 48, 53, 54, 55, 58), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 80077.0, 7.0, 20.0))","Map(vectorType -> dense, length -> 2, values -> List(6.762037683428458, -6.762037683428458))","Map(vectorType -> dense, length -> 2, values -> List(0.9988444673635481, 0.0011555326364518503))",0.0
17.0,?,104025.0,11th,7.0,Never-married,?,Own-child,White,Male,0.0,0.0,18.0,United-States,<=50K,5.0,0.0,7.0,2.0,3.0,1.0,0.0,"Map(vectorType -> sparse, length -> 5, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> 

また、SQLを用いることで、予測結果を年齢別、職業別に集計することができます。SQLを実行するために、予測結果のデータセットから一時ビューを作成します。

In [0]:
cvPredDF.createOrReplaceTempView("finalPredictions")

In [0]:
%sql
SELECT occupation, prediction, count(*) AS count
FROM finalPredictions
GROUP BY occupation, prediction
ORDER BY occupation

occupation,prediction,count
?,0.0,349
?,1.0,14
Adm-clerical,1.0,54
Adm-clerical,0.0,731
Armed-Forces,0.0,3
Craft-repair,1.0,72
Craft-repair,0.0,771
Exec-managerial,0.0,381
Exec-managerial,1.0,415
Farming-fishing,1.0,8


In [0]:
%sql
SELECT age, prediction, count(*) AS count
FROM finalPredictions
GROUP BY age, prediction
ORDER BY age

age,prediction,count
17.0,0.0,57
18.0,1.0,1
18.0,0.0,95
19.0,0.0,134
20.0,0.0,139
21.0,0.0,147
21.0,1.0,1
22.0,0.0,148
23.0,1.0,1
23.0,0.0,191
